In [1]:
import numpy as np
import math
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Class

## Affine

In [2]:
class Affine:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        
        self.optimizer = optimizer
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = initializer.init_W(n_nodes1, n_nodes2)
        self.B = initializer.init_B(n_nodes2)
        
        # Adagradで使用する各層の前回までの重み
        self.H_before_W = np.zeros_like(self.W)
        self.H_before_B = np.zeros_like(self.B)
    
    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """       
        # 逆伝播で使用するためインスタンス化
        self.X = X
        
        A = X @ self.W + self.B
        
        return A
    
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        self.dB = dA.sum(axis=0)
        self.dW = self.X.T @ dA
        dZ = dA @ self.W.T
        
        # 更新
        # ここの書き方がよくわかってない
        self = self.optimizer.update(self)
        
        return dZ

## SimpleInialzer

In [3]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
        
    def init_W(self, *shape):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W : 重みの初期値
        """
        W = np.random.randn(*shape) * self.sigma
        return W


    def init_B(self, *shape):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :　バイアスの初期値
        """
        B = np.zeros(*shape)
        return B

## SGD

In [4]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    
    def __init__(self, lr):
        self.lr = lr
    
    def update(self, layer):
        """
        Wの更新
        
        param
        --------------
        layer : 更新前の層のインスタンス
        --------------
        
        return
        ---------------
        layer : 更新後の層のインスタンス
        """
        
        # 引数のメンバ値を更新する
        layer.W -= self.lr * layer.dW 
        layer.B -= self.lr * layer.dB 
        
        return layer

## Sigmoid

In [5]:
class Sigmoid:
    """
    sigmoid関数
    """
    
    
    def forward(self, A):
        """
        順伝播
        """
        Z = 1 / (1 + np.exp(-A))
        
        # 逆伝播でつかうためインスタンス化
        self.Z = Z
        
        return Z
    
    
    def backward(self, dZ):
        """
        逆伝播
        """
        dA = dZ * (self.Z * (1 - self.Z))
        
        return dA
        

## tanh

In [6]:
class Tanh:
    """
    tanh関数
    """
    
    
    def forward(self, A):
        """
        順伝播
        """
        Z = np.tanh(A)
        
        # 逆伝播でつかうためインスタンス化
        self.Z = Z
        
        return Z
    
    
    def backward(self, dZ):
        """
        逆伝播
        """
        dA = dZ * (1 - self.Z ** 2)
        
        return dA
    

## Softmax

In [7]:
class Softmax:
    """
    softmax関数
    """
    
    
    def forward(self, A):
        """
        順伝播
        """
        Z = np.exp(A) / np.exp(A).sum(axis=1).reshape(-1, 1)
        
        # 逆伝播でつかうためインスタンス化
        self.Z = Z
        
        return Z
    
    
    def backward(self, y):
        """
        逆伝播
        """
        # 交差エントロピー誤差層の逆伝播+softmax層の逆伝播
        
        dA = 1 / y.shape[0] * (self.Z - y)
        
        return dA
  

## ReLU

In [8]:
class ReLU:
    """
    LeRU関数
    """
    
    
    def forward(self, A):
        """
        順伝播
        """
        self.A = A
        Z = np.maximum(0, A)
        
        return Z
    
    
    def backward(self, dZ):
        """
        逆伝播
        """
        dA = dZ * np.where(self.A > 0, 1, 0)
        
        return dA
        

## XavierInitializer

In [9]:
class XavierInitializer:
    
    
    def init_W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W : 重みの初期値
        """
        self.sigma = np.sqrt(1 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    
    def init_B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :　バイアスの初期値
        """
        B = self.sigma * np.random.randn(n_nodes2)
        return B

## HeInitializer

In [10]:
class HeInitializer:

    def init_W(self, n_nodes1):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W : 重みの初期値
        """
        self.sigma = math.sqrt(2 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W

    def init_B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :　バイアスの初期値
        """
        B = self.sigma * np.random.randn(n_nodes2)
        return B
    

## Adagrad

In [11]:
class AdaGrad:
    """
    各FC層の重み及びバイアスの更新
    
    param
    -------------
    lr : 学習率
    
    """
    
    def __init__(self, lr):
        self.lr = lr
        self.delta = 1e-7
    
    def update(self, layer):
        """
        Wの更新
        
        param
        --------------
        layer : FCクラスのインスタンス
        --------------
        
        return
        ---------------
        layer : FCクラスのインスタンス
        """
        
        
        hW = layer.H_before_W + (layer.dW * layer.dW)
        # h.shape = dw.shape = w.shape
        
        layer.W -= self.lr * layer.dW / np.sqrt(hW + self.delta)
        # 割り算なのでdW / hは形が変わらないはず(掛け算であればアダマール積を取る。)
        # そもそも行列割り算という概念がないため、割り算は同じ形じゃないとできない
        
        layer.H_before_W = hW
        # 次使う場合のためにhは更新する
                 
        hB = layer.H_before_B + (layer.dB * layer.dB)
        
        layer.B -= self.lr * layer.dB / np.sqrt(hB + self.delta)
        
        layer.H_before_B = hB
        # 次使う場合のためにhは更新する
        
        return layer

## getmnibatch

In [12]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        # ceilは切り上げ関数
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(int)
    
    # len関数が使われるとこの値を返す。
    def __len__(self):
        return self._stop
    
    # 要素をインスタンス変数に入れると値を返す
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]
    
    # iterが使われるとこの値を返す、なんでselfそのものを返す？？
    # そういう物っぽい、nextと組み合わせて使われる
    # for分繰り返されるという訳ではなく、forを行う前に一度だけ実行される
    def __iter__(self):
        self._counter = 0
        return self
    
    # forの回数実行される
    def __next__(self):
        # 要素の終わりまでいったら自動的に終了する
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

## DNN(Activete : ReLU)

In [ ]:
class ScratchDeepNeuralNetrowkClassifier:
    
    
    def __init__(self, n_nodes1, n_nodes2, n_output, epoch=20, lr=0.02, sigma=0.1, verbose = True):
        """
        self.sigma : ガウス分布の標準偏差
        self.lr : 学習率
        self.n_nodes1 : 1層目のノード数
        self.n_nodes2 : 2層目のノード数
        self.n_output : 出力層のノード数
        self.epoch : エポック数
        self.loss_train : 訓練データの損失
        self.loss_val : 検証データの損失
        self.verbose : 学習過程を表示するか
        """
        self.sigma = sigma
        self.lr = lr
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.n_output = n_output
        self.epoch = epoch
        self.loss_train = np.zeros(epoch)
        self.loss_val = np.zeros(epoch)
        self.verbose = verbose
        
        
    def get_loss(self, X, y_ture):
        """
        クロスエントロピー誤差を計算
        log(X)が最大値0の値を取るため-をかける必要がある。
        全て正解の場合0を取る。

        param
        -------------------
        X : 次の形のndarray(batch_size, n_features)
        入力値
        y_ture : 次の形のndarray(batch_size, n_class)
        正解ラベル

        return
        --------------------
        L : float(スカラー)
        """

        h = 1e-7      
        L = - np.sum(y_ture * np.log(X + h) / len(y_ture))
        return L
        
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
        X : 次の形のndarray, shape (n_samples, n_features)
        訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
        訓練データの正解値
        """
        
        # 1層目の入力特徴量数
        self.n_features = X.shape[1]
        
        # Affine, optimizer, initializerの決定
        # Affine :3層, optimizer : SGD, initializer : simple
        optimizer = SGD(self.lr)
        self.Affine1 = Affine(self.n_features, self.n_nodes1, SimpleInitializer(self.sigma), optimizer)
        self.activation1 = ReLU()
        self.Affine2 = Affine(self.n_nodes1, self.n_nodes2, SimpleInitializer(self.sigma), optimizer)
        self.activation2 = ReLU()
        self.Affine3 = Affine(self.n_nodes2, self.n_output, SimpleInitializer(self.sigma), optimizer)
        self.activation3 = Softmax()      
        
        # エポック毎に更新
        for i in range(self.epoch):
            
            # バッチ処理を実行
            get_mini_batch = GetMiniBatch(X, y, batch_size=20)
            
            #
            # バッチでループ
            #
            for mini_X_train, mini_y_train in get_mini_batch:

                # 順伝播
                A1 = self.Conv1d.forward(X)
                A1 = A1.reshape(A1.shape[0], A1.shape[-1])
                Z1 = self.activation1.forward(A1)
                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.FC3.forward(Z2)
                Z3 = self.activation3.forward(A3)

                # 逆伝播
                dA3 = self.activation3.backward(y_true) 
                dZ2 = self.FC3.backward(dA3)
                dA2 = self.activation2.backward(dZ2)
                dZ1 = self.FC2.backward(dA2)
                dA1 = self.activation1.backward(dZ1)
                dA1 = dA1[:, np.newaxis]
                dZ0 = self.Conv1d.backward(dA1) 
                
            #
            # 更新パラメータを使って全データで検証
            #
            A1 = self.Conv1d.forward(X)
            A1 = A1.reshape(A1.shape[0], A1.shape[-1])
            Z1 = self.activation1.forward(A1)
            A2 = self.FC2.forward(Z1)
            Z2 = self.activation2.forward(A2)
            A3 = self.FC3.forward(Z2)
            Z3 = self.activation3.forward(A3)
            
            # lossの計算
            self.loss_train[i] = self.get_loss(Z3, y)
            
            #verboseをTrueにした際は学習過程などを出力する
            if self.verbose:
                print('loss : {}'.format(self.loss_train[i]))
        
            # 検証データ
            if X_val is not None:
                #
                # 検証データの出力値を求める
                #
                A1 = self.Conv1d.forward(X)
                A1 = A1.reshape(A1.shape[0], A1.shape[-1])
                Z1 = self.activation1.forward(A1)
                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.FC3.forward(Z2)
                Z3 = self.activation3.forward(A3)
                
                # lossの計算
                self.loss_val[i] = self.get_loss(Z3, y_val) 
            
        
        
    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル
        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        A1 = self.Conv1d.forward(X)
        A1 = A1.reshape(A1.shape[0], A1.shape[-1])
        Z1 = self.activation1.forward(A1)
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)

        # 最も大きいインデックスをクラスとして採用
        return np.argmax(Z3, axis=1)

## SimpleConv1d (問題1)

In [ ]:
class SimpleConv1d():
    """
    チャンネル数１の畳み込み層
    
    param
    --------------------
    self.W_size : ndarray (W_size, )
    フィルタサイズ(重みのサイズ？？？？)
    self.padding : int
    パディングのサイズ
    self.storide : int
    ストライドのサイズ
    
    """
    
    def __init__(self, W_size, optimizer, padding=0, storide=1):
        
        self.optimizer = optimizer
        self.padding = padding
        self.storide = storide
        
        # Wの初期化(1次元のため、初期化クラスは使えない)
        self.W = np.array([3, 5, 7], dtype=float)
        #self.W = np.random.randn(W_size)
        
        # Bの初期化
        self.B = np.array([1], dtype=float)
        
        
    def get_N_out(self, N_in, padding=0, storide=1):
        """
        畳み込みを行った後の特徴量の数
        """

        # npではないのでpython組み込み関数を使用
        N_out = int((N_in + 2 * self.padding - self.W.shape[0]) / self.storide + 1)

        return N_out

        
    def forward(self, X):
        """
        順伝播
        
        param
        --------------
        self.X : ndarray (n_features, )
        入力サイズ
        """
        
        # 逆伝播で使用する
        self.X = X
        
        # 出力のサイズ指定
        A = np.zeros(self.get_N_out(X.shape[0]))
        
        # aの値を更新
        for i in range(A.shape[0]):
            A[i] = int(X[i: i + self.W.shape[0]] @ self.W + self.B)

        A = A.astype(int)
        self.A = A
        
        return A
        
    
    def backward(self, dA):
        """
        逆伝播
        """
        
        # 初期化(毎回行う)
        dW = np.zeros_like(self.W)
        dB = np.zeros_like(self.B)

        # 誤差の計算
        for i in range(self.A.shape[0]):
            dW = dW + dA[i] * self.X[i : i+self.W.shape[0]]
            dB = dB + dA[i]
        
        
        # 初期化
        dX = np.zeros_like(self.X)

        # 誤差の計算
        for i in range(dX.shape[0]):
            for s in range(self.W.shape[0]):
                # 入力の枠外の微分は0とする(index,shapeの数え方が違うので-1)
                if (i-s < 0) or (i-s > dA.shape[0]-1):
                    pass
                # 枠内の微分値は足し合わせる(重みが縦横反転する)
                else:
                    dX[i] += dA[i-s] * self.W[s]
                    
                    
        # 更新
        self.dW = dW
        self.dB = dB
        self = self.optimizer.update(self)

        return dX

In [ ]:
x = np.array([1,2,3,4])
test = SimpleConv1d(W_size=3, optimizer=SGD(lr=0.01))
test.forward(x)

In [ ]:
delta_a = np.array([10, 20])
test.backward(delta_a)

## Conv1d (問題4)

![Imgur](https://i.imgur.com/EtJ2ZdWl.jpg)
![Imgur](https://i.imgur.com/SICodEJl.jpg)
![Imgur](https://i.imgur.com/gtU9vzZl.jpg)

In [17]:
class Conv1d():
    """
    チャンネル数１の畳み込み層
    
    param
    --------------------
    self.W_size : ndarray (W_size, )
    フィルタサイズ(重みのサイズ？？？？)
    self.padding : int
    パディングのサイズ
    self.storide : int
    ストライドのサイズ
    
    """
    
    def __init__(self, n_input, n_filiter, n_input_ch, n_output_ch, optimizer, initializer, padding=0, storide=1):
        
        self.n_input = n_input
        self.n_filiter = n_filiter
        self.optimizer = optimizer
        self.n_input_ch = n_input_ch
        self.n_output_ch = n_output_ch
        self.W = initializer.init_W(n_output_ch, n_input_ch, n_filiter)
        self.B = initializer.init_B(n_output_ch)
        self.padding = padding
        self.storide = storide
        
        # 出力サイズの計算
        self.n_output = int((n_input + 2 * padding - n_filiter) / storide + 1)
        
        # Adagradで使用する各層の前回までの重み
        self.H_before_W = np.zeros_like(self.W)
        self.H_before_B = np.zeros_like(self.B)

        
    def forward(self, X):
        """
        順伝播
        
        param
        --------------
        self.X : ndarray (n_features, )
        入力サイズ
        """ 
        
        
#自作モデル、for文のため見送り


#         # 形を作成(出力チャンネル、　出力ノード)
#         A = np.zeros((3, 2))
#         # Aの行を指定
#         for i in range(A.shape[0]):
#             # Aの列を指定
#             for j in range(A.shape[1]):
#                 # Xのチャンネルを指定
#                 for k in range(X.shape[0]):
#                     # Xの形をWが存在するところまでに切り取る(Wにゼロを追加するのと同義)
#                     A[i, j] += X[k, j : j+w.shape[2]].T @ W[i, j]
#                 # バイアスの計算
#                 A[i, j] += b[i]

#         A = A.astype(int)

#         return A
        
        
        # バッチ数
        self.n_samples = X.shape[0]
        
        # 計算のために逆転させる
        # 逆転させているとあるが、多分逆転してない
        X = X.reshape(self.n_samples, self.n_input_ch, self.n_input)
        
        # Xの特徴量の左右に0埋め実施
        # Aの形が(3, 2)のためXの特徴量方向に３パターンスライドさせてあげる必要がある。
        # paddingが0であれば、そのまま０を使わないで解けるがpaddingがあると4, 6と増えていく。
        self.X = np.pad(X, ((0,0), (0,0), ((self.n_filiter-1), 0)))
        
        # 出力配列（A）の計算のためゼロ配列X1を用意する
        # サンプル毎にXをスライドさせたものをフィルタ数分用意する。
        # 上で左右+1しているのでX+2の特徴量を持つ。
        # Xをスライドさせることで、convを行っているイメージ
        self.X1 = np.zeros((self.n_samples, self.n_input_ch, self.n_filiter, self.n_input+(self.n_filiter-1)))

        # 重みの長さでループ
        for i in range(self.n_filiter):
            # ずらしながら上書き
            self.X1[:, :, i] = np.roll(self.X, -i, axis=-1)
            
        # 重みとバイアスを考慮して計算
        # バカむずい
        # 一度5次元まで拡張して、３次元方向に足し合わせてる。
        # 自分の描いたfor文のほうがわかるが、計算がこちらの方が楽
        # 足し合わせる方向は、サンプル数とフィルタ数方向
        A = np.sum(self.X1[:, np.newaxis, :, :, self.n_filiter-1-self.padding:self.n_input+self.padding:self.storide]*self.W[:, :, :, np.newaxis], axis=(2, 3)) + self.B.reshape(-1,1)
        return A
        
    
    def backward(self, dA):
        """
        逆伝播
        """
        
# 自作記念モデル
#
#         # 初期化(毎回行う)
#         dW = np.zeros_like(self.W)
#         dB = np.zeros_like(self.B)

#         # 誤差の計算
#         for i in range(dW.shape[0]):
#             for j in range(dW.shape[1]):
#                 for l in range(dA.shape[1]):
#                     dW[i, j] += dA[i, l] * self.X[j, l : l+dW.shape[2]]
#                     if j == 0:
#                         dB[i] += dA[i, l]
        
        
#         # 初期化
#         dX = np.zeros_like(self.X)

#         # 誤差の計算
#         for i in range(dX.shape[0]):
#             for j in range(dX.shape[1]):
#                 for k in range(dA.shape[0]):
#                     for s in range(self.W.shape[2]):
#                         # 入力の枠外の微分は0とする
#                         if (j-s < 0) or (j-s > dA.shape[1]-1):
#                             pass
#                         # 枠内の微分値は足し合わせる
#                         else:
#                             dX[i, j] += dA[k, j-s] * self.W[k, i, s]

        # 重みの勾配
        self.dW = np.sum(dA[:, :, np.newaxis, np.newaxis]*self.X1[:, np.newaxis, :, :, self.n_filiter-1-self.padding:self.n_input+self.padding:self.storide], axis=(0, -1))
        # バイアスの勾配
        self.dB = np.sum(dA, axis=(0, -1))
        # 逆伝播の値計算のためにdAを変形
        self.dA = np.pad(dA, ((0,0), (0,0), (0, (self.n_filiter-1))))
        # 出力配列（dX）の計算のためゼロ配列dA1を用意する
        self.dA1 = np.zeros((self.n_samples, self.n_output_ch, self.n_filiter, self.dA.shape[-1]))
        # 重みの長さでループ
        for i in range(self.n_filiter):
            self.dA1[:, :, i] = np.roll(self.dA, i, axis=-1)
        dX = np.sum(self.W[:, :, :, np.newaxis]*self.dA1[:, :, np.newaxis], axis=(1,3))

                     
        # 更新
        self = self.optimizer.update(self)

        return dX

In [18]:
test = Conv1d(n_input=4, n_filiter=3, n_input_ch=2, n_output_ch=3, initializer=SimpleInitializer(sigma=0.01), optimizer=SGD(lr=0.01))
test.W

array([[[ 0.00109616, -0.01927108,  0.00827651],
        [ 0.00205352, -0.00880943,  0.00459448]],

       [[ 0.00465457, -0.00263397,  0.01836861],
        [ 0.00858373,  0.00207386, -0.01462086]],

       [[ 0.01260558, -0.00184931, -0.0074794 ],
        [ 0.00461731, -0.00387589, -0.01238062]]])

In [19]:
test.forward(X=np.array([[[1, 2, 3, 4], [2, 3, 4, 5]],[[1, 2, 3, 4], [2, 3, 4, 5]]]))

array([[[-0.01655981, -0.02861966],
        [ 0.01939807,  0.03582401],
        [-0.06544679, -0.07380913]],

       [[-0.01655981, -0.02861966],
        [ 0.01939807,  0.03582401],
        [-0.06544679, -0.07380913]]])

## ScratchCNNClassifier (問題5-7)

In [20]:
class ScratchCNNClassifier:
    
    
    def __init__(self, n_nodes2, n_output, n_filiter, epoch=20, lr=0.02, sigma=0.1, verbose = True, Activater=Tanh, Optimizer=AdaGrad):
        """
        self.sigma : ガウス分布の標準偏差
        self.lr : 学習率
        self.n_nodes2 : 2層目のノード数
        self.n_output : 出力層のノード数
        self.epoch : エポック数
        self.loss_train : 訓練データの損失
        self.loss_val : 検証データの損失
        self.verbose : 学習過程を表示するか
        """
        self.sigma = sigma
        self.lr = lr
        self.n_nodes2 = n_nodes2
        self.n_output = n_output
        self.n_filiter = n_filiter
        self.epoch = epoch
        self.loss_train = np.zeros(epoch)
        self.loss_val = np.zeros(epoch)
        self.verbose = verbose
        self.Activater = Activater
        if Activater == Sigmoid or Activater == Tanh:
            self.Initializer = XavierInitializer
        elif Activater == ReLU:
            self.Initializer = HeInitializer
        self.Optimizer = Optimizer
        
        
    def get_loss(self, X, y_ture):
        """
        クロスエントロピー誤差を計算
        log(X)が最大値0の値を取るため-をかける必要がある。
        全て正解の場合0を取る。

        param
        -------------------
        X : 次の形のndarray(batch_size, n_features)
        入力値
        y_ture : 次の形のndarray(batch_size, n_class)
        正解ラベル

        return
        --------------------
        L : float(スカラー)
        """

        h = 1e-7      
        L = - np.sum(y_ture * np.log(X + h) / len(y_ture))
        return L
        
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
        X : 次の形のndarray, shape (n_samples, n_features)
        訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
        訓練データの正解値
        """
        
        # 1層目の入力特徴量数
        self.n_features = X.shape[1]
        
        #
        # フィルターの数はハイパーパラメータ
        #
        
        # 784→778 Convなので初期化はシンプルなやつ
        self.Conv1d = Conv1d(n_input=self.n_features, n_filiter=self.n_filiter, n_input_ch=1, n_output_ch=1, initializer=SimpleInitializer(self.sigma), optimizer=self.Optimizer(self.lr))
        self.activation1 = ReLU()
        # 778→400
        self.Affine2 = Affine(n_nodes1=self.Conv1d.n_output, n_nodes2=self.n_nodes2, initializer=self.Initializer(), optimizer=self.Optimizer(self.lr))
        self.activation2 = ReLU()
        # 400→10
        self.Affine3 = Affine(n_nodes1=self.n_nodes2, n_nodes2=self.n_output, initializer=self.Initializer(), optimizer=self.Optimizer(self.lr))
        self.activation3 = Softmax()      
        
        # エポック毎に更新
        for i in range(self.epoch):
            
            # バッチ処理を実行
            get_mini_batch = GetMiniBatch(X, y, batch_size=20)
            
            #
            # バッチでループ
            #
            for mini_X_train, mini_y_train in get_mini_batch:

                # 順伝播
                A1 = self.Conv1d.forward(mini_X_train)
                A1 = A1.reshape(A1.shape[0], A1.shape[-1])
                Z1 = self.activation1.forward(A1)
                A2 = self.Affine2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.Affine3.forward(Z2)
                Z3 = self.activation3.forward(A3)

                # 逆伝播
                dA3 = self.activation3.backward(mini_y_train) 
                dZ2 = self.Affine3.backward(dA3)
                dA2 = self.activation2.backward(dZ2)
                dZ1 = self.Affine2.backward(dA2)
                dA1 = self.activation1.backward(dZ1)
                dA1 = dA1[:, np.newaxis]
                dZ0 = self.Conv1d.backward(dA1) 
                
            #
            # 更新パラメータを使って全データで検証
            #
            A1 = self.Conv1d.forward(X)
            A1 = A1.reshape(A1.shape[0], A1.shape[-1])
            Z1 = self.activation1.forward(A1)
            A2 = self.Affine2.forward(Z1)
            Z2 = self.activation2.forward(A2)
            A3 = self.Affine3.forward(Z2)
            Z3 = self.activation3.forward(A3)
            
            # lossの計算
            self.loss_train[i] = self.get_loss(Z3, y)
            
            #verboseをTrueにした際は学習過程などを出力する
            if self.verbose:
                print('loss : {}'.format(self.loss_train[i]))
        
            # 検証データ
            if X_val is not None:
                #
                # 検証データの出力値を求める
                #
                A1 = self.Conv1d.forward(X)
                A1 = A1.reshape(A1.shape[0], A1.shape[-1])
                Z1 = self.activation1.forward(A1)
                A2 = self.Affine2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.Affine3.forward(Z2)
                Z3 = self.activation3.forward(A3)
                
                # lossの計算
                self.loss_val[i] = self.get_loss(Z3, y_val) 
            
        
        
    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル
        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        A1 = self.Conv1d.forward(X)
        A1 = A1.reshape(A1.shape[0], A1.shape[-1])
        Z1 = self.activation1.forward(A1)
        A2 = self.Affine2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.Affine3.forward(Z2)
        Z3 = self.activation3.forward(A3)

        # 最も大きいインデックスをクラスとして採用
        return np.argmax(Z3, axis=1)

## データ呼び出し

In [21]:
# データ読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 画像データ→行データに
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# 正規化
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# onehotベクトル化
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

# 訓練データと評価データに
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

/var/folders/ll/cdp3xh194t7_9c631lm07yx00000gn/T/xpython_25385/797439057.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train = X_train.astype(np.float)
/var/folders/ll/cdp3xh194t7_9c631lm07yx00000gn/T/xpython_25385/797439057.py:10: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test = X_test.astype(np.float)


## データテスト (問題8)

In [111]:
test_15 = ScratchCNNClassifier(n_nodes2=200, n_output=10, n_filiter=15)
test_15.fit(X_train_[:1000], y_train_[:1000])

loss : 0.3474151445104263
loss : 0.18587008337945998
loss : 0.11415309160752884
loss : 0.07001890594241297
loss : 0.045121322570963664
loss : 0.02996402590214832
loss : 0.020527056716755886
loss : 0.014933895472887551
loss : 0.01145086405366445
loss : 0.009143521155446557
loss : 0.007572240212282073
loss : 0.006428984475571742
loss : 0.005561779615762482
loss : 0.004869386932522326
loss : 0.004321815374433001
loss : 0.003871053907142638
loss : 0.0034966105996420247
loss : 0.0031794556754198918
loss : 0.002913300005262992
loss : 0.002681960378377793


In [115]:
y_pred = test_15.predict(X_test)
accuracy_score(y_test, y_pred)

0.8894

In [119]:
test_2 = ScratchCNNClassifier(n_nodes2=200, n_output=10, n_filiter=2)
test_2.fit(X_train_[:1000], y_train_[:1000])

loss : 0.3747987017143381
loss : 0.22858483357411552
loss : 0.15288131796792997
loss : 0.10513262647960298
loss : 0.07353838155662988
loss : 0.05260812322199335
loss : 0.03876642076114238
loss : 0.028782460129756587
loss : 0.022124131264524914
loss : 0.017247272117804677
loss : 0.013910697369034452
loss : 0.011535443754460296
loss : 0.009740150693157871
loss : 0.008391364823601945
loss : 0.007331700517399073
loss : 0.006490631197494703
loss : 0.005801263539624252
loss : 0.005227795188655701
loss : 0.004758387739044469
loss : 0.004352409635734007


In [123]:
y_pred = test_2.predict(X_test)
accuracy_score(y_test, y_pred)

0.8975